# Homework 4.3

## 1.
Run ChIPQC for:
https://drive.google.com/open?id=1dKLt_m_vnHjNVo-1qoACu9GZZC9OBa9v  
(Links to an external site.)

__EFF_H3K4Me1_1_srt_dupr_chr11.bam__  
__EFF_H3K4Me1_2_srt_dupr_chr11.bam__  
__EFF_Input_srt_dupr_chr11.bam (Control)__  

### - Sort and index bam file  

In [1]:
%%bash
cd /Users/sfchen/Documents/2018_Su/aligned_bam_homework_1

printf "EFF_H3K4Me1_1_srt_dupr_chr11\nEFF_H3K4Me1_2_srt_dupr_chr11\nEFF_Input_srt_dupr_chr11\n" > EFF.txt

while read FILE; do

    # Sort the bam file
    samtools sort -o ${FILE}.srt.bam ${FILE}.bam

    # Index the bam file
    samtools index ${FILE}.srt.bam

done < EFF.txt

### - Run MACS2  

__NOTE:__ `macs2 -c/--control` The control or mock data file.

In [2]:
%%bash
cd /Users/sfchen/Documents/2018_Su/aligned_bam_homework_1

printf "EFF_H3K4Me1_1_srt_dupr_chr11\nEFF_H3K4Me1_2_srt_dupr_chr11\n" > sample.txt

while read FILE; do

    # MACS2 peak calling
    macs2 callpeak \
        -t ${FILE}.srt.bam \
        -c EFF_Input_srt_dupr_chr11.bam \
        -f BAM -n ${FILE}.srt \
        -g mm \
        -p 1e-2 \
        --broad \
        --nomodel \
        --bdg \
        --shift 0 \
        --extsize 147 \
        --keep-dup all &> ${FILE}.srt.MACS2.out

done < sample.txt

### - Create sample sheet  

__NOTE:__ According to [ChIPQC Manual](https://bioconductor.riken.jp/packages/3.1/bioc/manuals/ChIPQC/man/ChIPQC.pdf), provide control file path to "bamControl" column.  

`bamReads: file path for bam file containing aligned reads for ChIP sample`  
`bamControl: file path for bam file containing aligned reads for control sample.`

In [3]:
%%bash
cat /Users/sfchen/Documents/2018_Su/aligned_bam_homework_1/ChIPQC_samplesheet_EFF.csv

SampleID,Tissue,Factor,Condition,Replicate,bamReads,ControlID,bamControl,Peaks,PeakCaller
H3K4Me1_1,,,,1,EFF_H3K4Me1_1_srt_dupr_chr11.srt.bam,Input,EFF_Input_srt_dupr_chr11.srt.bam,EFF_H3K4Me1_1_srt_dupr_chr11.srt_peaks.broadPeak,macs
H3K4Me1_2,,,,2,EFF_H3K4Me1_2_srt_dupr_chr11.srt.bam,Input,EFF_Input_srt_dupr_chr11.srt.bam,EFF_H3K4Me1_2_srt_dupr_chr11.srt_peaks.broadPeak,macs

### - ChIPQC  

__NOTE:__ Check "4.3_Homework_R.ipynb"

## 2. Use Homer to search for TF motifs in broadPeak file (from Q1, EFF_H3K4Me1_1_srt_dupr_chr11.bam); genome reference: genome_chr11.fa


In [4]:
%%bash
cd /Users/sfchen/Documents/2018_Su/aligned_bam_homework_1
bed_name=EFF_H3K4Me1_1_srt_dupr_chr11.srt_peaks.broadPeak
chr11_genome=/Users/sfchen/Documents/2018_Su/UCSC_mm10/genome_chr11.fa
out_name=EFF_H3K4Me1_1_srt_dupr_chr11_HomerMF
err_name=EFF_H3K4Me1_1_srt_dupr_chr11_HomerMF.out
findMotifsGenome.pl $bed_name $chr11_genome $out_name &> $err_name

### - What are the top 5 knownResults?

In [7]:
%%bash
cd /Users/sfchen/Documents/2018_Su/aligned_bam_homework_1/EFF_H3K4Me1_1_srt_dupr_chr11_HomerMF
head -6 knownResults.txt | cut -f 1

Motif Name
ETV1(ETS)/GIST48-ETV1-ChIP-Seq(GSE22441)/Homer
Fli1(ETS)/CD8-FLI-ChIP-Seq(GSE20898)/Homer
ERG(ETS)/VCaP-ERG-ChIP-Seq(GSE14097)/Homer
ETS1(ETS)/Jurkat-ETS1-ChIP-Seq(GSE17954)/Homer
GABPA(ETS)/Jurkat-GABPa-ChIP-Seq(GSE17954)/Homer


### - What is the difference between knownResults and homerResults?

The top 5 best match of homerResults (aka. Homer *de novo* motif enrichment results) are:  

1. eyg/dmmpmm(Bergman)/fly(0.656)  
2. FMR1(KH)/Homo_sapiens-RNCMPT00016-PBM/HughesRNA(0.712)  
3. TEC1(MacIsaac)/Yeast(0.707)  
4. LIN28A(CSD)/Homo_sapiens-RNCMPT00162-PBM/HughesRNA(0.714)  
5. FZF1/MA0298.1/Jaspar(0.667)

According to the [Homer manual](http://homer.ucsd.edu/homer/introduction/basics.html):
- *De novo* motif discovery allows to directly query the sequence to discover which motifs are the MOST enriched sequences in target set. Best saved for last.  
- Known motif discovery will simply tell which of the known motifs is most enriched in target set. Also, it's performed first since it is usually faster, and gives a faster look at what's enriched in your target regions.


__Supplement:__ stdout of Homer.

In [8]:
%%bash
cat /Users/sfchen/Documents/2018_Su/aligned_bam_homework_1/EFF_H3K4Me1_1_srt_dupr_chr11_HomerMF.out


	Position file = EFF_H3K4Me1_1_srt_dupr_chr11.srt_peaks.broadPeak
	Genome = /Users/sfchen/Documents/2018_Su/UCSC_mm10/genome_chr11.fa
	Output Directory = EFF_H3K4Me1_1_srt_dupr_chr11_HomerMF
	Using Custom Genome
	Peak/BED file conversion summary:
		BED/Header formatted lines: 17844
		peakfile formatted lines: 0

	Peak File Statistics:
		Total Peaks: 17844
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Background files for 200 bp fragments found.

	Extracting sequences from file: /Users/sfchen/Documents/2018_Su/UCSC_mm10/genome_chr11.fa
	Looking for peak sequences in a single file (/Users/sfchen/Documents/2018_Su/UCSC_mm10/genome_chr11.fa)
	Extracting 17844 sequences from chr11

	Not removing redundant sequences


	Sequences processed:
		Auto detected maximum sequence length of 201 bp
		17844 